In [ ]:
import requests
import json
import pandas as pd
import altair as alt

In [ ]:
# Instituts-Kürzel
acronym = "ISI"

# Jahr von (min) bis (max). Für die Einschränkung auf ein Jahr, bitte den gleichen Wert für date_min und date_max einfügen
#date = "2021"
date_min = "2020" 
date_max = "2022"


url = "https://publica-rest.fraunhofer.de/server/api/discover/search/objects?query=institute:("+acronym+")%20AND%20(dc.relation.ispartof:*)&configuration=researchoutputs&f.dateIssued=%5B"+date_min+"%20TO%20"+date_max+"%5D,equals"

r = requests.request("GET", url)
data = r.text
data = json.loads(data)
#print(data)
anzahl = data["_embedded"]["searchResult"]["page"]["totalElements"]
anzahl = int(anzahl)
print(anzahl)
pages = data["_embedded"]["searchResult"]["page"]["totalPages"]
pages = int(pages)
#print(pages)
object = data["_embedded"]["searchResult"]["_embedded"]["objects"]

pub_uuid = []
pub_name = []
pub_href = []
pub_journal = []
pub_journalAuth = []

object = data["_embedded"]["searchResult"]["_embedded"]["objects"]

for j in range(len(object)):
    pub_uuid.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"])
    pub_name.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["name"])
    pub_href.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_links"]["indexableObject"]["href"])
    pub_journal.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["value"])
    pub_journalAuth.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["authority"])

    
if pages > 1:
    for l in range(pages-1):
        url = data["_embedded"]["searchResult"]["_links"]["next"]["href"]
        r = requests.request("GET", url)
        #print(l)
        data = r.text
        data = json.loads(data)
        object = data["_embedded"]["searchResult"]["_embedded"]["objects"]
        for j in range(len(object)):
            pub_uuid.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"])
            pub_name.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["name"])
            pub_href.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_links"]["indexableObject"]["href"])
            pub_journal.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["value"])
            pub_journalAuth.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["authority"])
           


378


In [ ]:
df = pd.DataFrame({'uuid':pub_uuid})
df['name'] = pub_name
df['href'] = pub_href
df['journal'] = pub_journal
df['journalAuth'] = pub_journalAuth

#print(df)

value_counts = df['journal'].value_counts()
print(value_counts)

Technological forecasting and social change          16
Energy policy                                        15
Environmental innovation and societal transitions    14
Energy research & social science                     10
Scientometrics                                       10
                                                     ..
Regions eZine. Online journal                         1
Die Hochschule                                        1
Qualität und Zuverlässigkeit : QZ                     1
KronoScope                                            1
Economics of energy & environmental policy            1
Name: journal, Length: 195, dtype: int64


In [ ]:
value_counts = df['journal'].value_counts()
df_journal = pd.DataFrame(value_counts.reset_index())
df_journal.columns = ['Journal', 'Anzahl']
df_journal

,Journal,Anzahl
0,Technological forecasting and social change,16
1,Energy policy,15
2,Environmental innovation and societal transitions,14
3,Energy research & social science,10
4,Scientometrics,10
...,...,...
190,Regions eZine. Online journal,1
191,Die Hochschule,1
192,Qualität und Zuverlässigkeit : QZ,1
193,KronoScope,1


In [ ]:
bars=alt.Chart(df).mark_bar().encode(
x='count(journal):Q',
y=alt.Y("year:O")
)
text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3 
).encode(
    text='count(journal):Q'
)
(bars + text).properties(
    title='Contributions per Journal'
)

alt.LayerChart(...)

In [ ]:
value_counts = df['journal'].value_counts(dropna=True, sort=True).nlargest(10) # Einschränkung auf die Top Ten.
df_top10 = pd.DataFrame(value_counts.reset_index())
df_top10.columns = ['Journal', 'Anzahl']

df_top10

,Journal,Anzahl
0,Technological forecasting and social change,16
1,Energy policy,15
2,Environmental innovation and societal transitions,14
3,Energy research & social science,10
4,Scientometrics,10
5,Zeitschrift für Technikfolgenabschätzung in Th...,8
6,Sustainability,8
7,Energy strategy reviews,7
8,The journal of technology transfer,7
9,Energies,6


In [ ]:

bars=alt.Chart(df_top10).mark_bar().encode(
x='Anzahl:Q',
y=alt.Y('Journal:N', sort='-x')
)
text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text='Anzahl:Q'
)
(bars + text).properties(
    title='Top 10 Journals'
)

alt.LayerChart(...)